In [ ]:
!pip install -q ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
from IPython.display import display, Javascript
import PIL
import numpy as np
from base64 import b64decode, b64encode
import io

# Webcam

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  return img_rgb

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
def detect_color(img, Threshold=0.01):
    """
    detect red and yellow
    :param img:
    :param Threshold:
    :return:
    """

    desired_dim = (30, 90)  # width, height
    img = cv2.resize(np.array(img), desired_dim, interpolation=cv2.INTER_LINEAR)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    # Define red color range
    lower_red = np.array([0, 100, 50])
    upper_red = np.array([4, 255, 255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

    lower_red1 = np.array([170, 100, 50])
    upper_red1 = np.array([180, 255, 255])
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)

    mask_red = mask0 + mask1
    # Define yellow color range
    lower_yellow = np.array([14, 100, 50])
    upper_yellow = np.array([39, 255, 255])
    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)

    # Define green color range
    lower_green = np.array([56, 100, 50])
    # upper_green = np.array([79, 255, 255])
    upper_green = np.array([89, 255, 255])
    mask_green = cv2.inRange(img_hsv, lower_green, upper_green)

    color_mask = mask_red + mask_yellow + mask_green

    # Show each mask result
    target_red = cv2.bitwise_and(img, img, mask=mask_red)
    target_yellow = cv2.bitwise_and(img, img, mask=mask_yellow)
    target_green = cv2.bitwise_and(img, img, mask=mask_green)

    # Compare the percentage of red, yellow, and green values
    rate = np.count_nonzero(color_mask) / (desired_dim[0] * desired_dim[1])

    # Show number of pixels of each color (Debugging)
    red_pixels = np.count_nonzero(mask_red)
    yellow_pixels = np.count_nonzero(mask_yellow)
    green_pixels = np.count_nonzero(mask_green)

    if rate > Threshold:
        # Check the dominant color
        if red_pixels > yellow_pixels and red_pixels > green_pixels:
            return ("Red", (255, 0 ,0))
        elif yellow_pixels > red_pixels and yellow_pixels > green_pixels:
            return ("Yellow", (255, 255, 0))
        elif green_pixels > red_pixels and green_pixels > yellow_pixels:
            return ("Green", (0, 255, 0))
        else:
            return ("Unknown", (0, 0, 0))
    else:
        return ("None", (0, 0, 0))

# Model

In [ ]:
model = YOLO('yolov8n.pt')

### Colab

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'

bbox = ''
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = PIL.Image.fromarray(js_to_image(js_reply["img"]))

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    results = model.predict(img, verbose=False, classes=9, stream_buffer=True)
    # Process the results if needed
    for r in results:
        for i in range(len(r.boxes.cls)):
            x1,y1,x2,y2 = r.boxes.xyxy[i]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            crop_img = img.crop((x1, y1, x2, y2))
            col_text,color = detect_color(crop_img)
            bbox_array = cv2.rectangle(bbox_array, (x1,y1), (x2,y2), color, 2)
            cv2.putText(bbox_array, col_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes